# 2.4 Playground - design your own peptides and modifications and predict their retention time (optional for a later time for the real enthusiasts)

## 2.4.1 Make predictions for your own peptide and modifications combos

Provide the data for peptides you want to predict:

In [ ]:
#IIVINTPNNPIGK
dict_effect_aa = {
    "seq" : ["IIVINKPNNPIGK", "IIVINTPNNPIGK", "IIVINAPNNPIGK", "IIVINWPNNPIGK"],
    "modifications" : ["","","",""],
    "tr" : [0,1,2,3]
}

df_effect_aa = pd.DataFrame(dict_effect_aa)

In [ ]:
preds = dlc.make_preds(seq_df=df_effect_aa)

Lets have a look at their predictions:

In [ ]:
plt.scatter(df_effect_aa.index,preds)
plt.xticks(df_effect_aa.index,df_effect_aa["seq"])
plt.ylabel("Predicted retention time (s)")
plt.show()

Provide the data for peptides+modifications you want to predict:

In [ ]:
#IIVINTPNNPIGK
dict_effect_aa = {
    "seq" : ["IIVINCPNNPIGK", "IIVINCPNNPIGK", "IIVINQPNNPIGK", "IIVINQPNNPIGK", "IIVINMPNNPIGK", "IIVINMPNNPIGK"],
    "modifications" : ["","6|carbamidomethyl","","6|Deamidated","","6|Formyl"],
    "tr" : [0,1,2,3,4,5]
}

df_effect_aa = pd.DataFrame(dict_effect_aa)

In [ ]:
preds = dlc.make_preds(seq_df=df_effect_aa)

In [ ]:
plt.scatter(df_effect_aa.index,preds)
plt.xticks(df_effect_aa.index,df_effect_aa["seq"]+"+"+df_effect_aa["modifications"],rotation=90)
plt.ylabel("Predicted retention time (s)")
plt.show()

## 2.4.2 Questions - playground retention time prediction

<ol>
  <li>Can you design a peptide that falls in between "IIVINKPNNPIGK" and "IIVINTPNNPIGK" in terms of retention time?</li>
  <li>What effect do certain modifications have? Is this expected?</li>
  <li>Do you expect that modifications always have the same effect?</li>
</ol>